In [276]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
from statsmodels.iolib.summary2 import summary_col
from statsmodels import sandbox as sb
from statsmodels.sandbox.regression import gmm as g

#### Results:

In [317]:
print(summary_of_NAC)
print(SimpleTable(summl.values.round(4),list(summl.columns),list(summl.index), title = 'Mean for Compliers for each Covariate'))

The Fractions of Compliers, Always Takers and Never Takers In Months: 15,20,24,48
               ft15   ft20   ft24   ft48 
-----------------------------------------
Never Takers  0.7154 0.7245 0.7324 0.7158
Always Takers 0.1472 0.1609 0.1602 0.2337
Compliers     0.1374 0.1146 0.1074 0.0505
-----------------------------------------
     Mean for Compliers for each Covariate     
                     ft15   ft20   ft24   ft48 
-----------------------------------------------
age35p              0.2775 0.292  0.2842 0.2891
agelt25             0.179  0.183  0.1925 0.1943
anykidsu6           0.5257 0.5281 0.5336 0.5268
hsgrad              0.5844 0.589  0.5953 0.6037
imm                 0.1088 0.1223 0.1224 0.1224
nevermarried        0.4798 0.4817 0.4809 0.5007
working_at_baseline 0.3663 0.3573 0.3521 0.3155
-----------------------------------------------


1)



In [278]:
df = pd.read_csv('welfare.csv')

In [279]:
X = df[['imm', 'hsgrad','agelt25','age35p','working_at_baseline','anykidsu6','nevermarried']]

In [280]:
df.head()

,imm,hsgrad,agelt25,age35p,treatment,working_at_baseline,anykidsu6,nevermarried,ft15,ft20,ft24,ft48,welfare15,welfare20,welfare24,welfare48
0,0,0,0,0,1,0,1,1,0,0.0,0.0,0.0,1,1,1,1
1,0,1,0,1,1,0,0,1,1,1.0,1.0,NaN,0,0,0,0
2,0,0,0,0,1,0,0,1,0,0.0,0.0,0.0,1,1,1,0
3,0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,1,0,0,0
4,0,1,0,0,0,0,1,1,0,0.0,0.0,0.0,1,1,1,1


In [281]:
control_proportion = df[df['treatment'] == 0].mean()[['ft15','ft20','ft24','ft48']]
treatment_proportion =  df[df['treatment'] == 1].mean()[['ft15','ft20','ft24','ft48']]


In [282]:
proportion = np.vstack([df[df['treatment'] == 0].mean()[['ft15','ft20','ft24','ft48']],\
                                df[df['treatment'] == 1].mean()[['ft15','ft20','ft24','ft48']]]).round(4)


col = ['ft15', 'ft20', 'ft24','ft48']
row = ['control', "treatment"]

summary_second_stage3 = SimpleTable(proportion,col, row, txt_fmt=default_txt_fmt)
print(summary_second_stage3)

           ft15   ft20   ft24   ft48 
-------------------------------------
control   0.1472 0.1609 0.1602 0.2337
treatment 0.2846 0.2755 0.2676 0.2842
-------------------------------------


In [283]:
never_takers = 1 - proportion[1].T
always_takers = proportion[0]
compliers = proportion[1].T - proportion[0].T

Write about the probablities of always takers, compliers and never takers. 

In [284]:
summary_fractions = np.vstack([[never_takers], [always_takers],[compliers]]).round(4)
col = ['ft15', 'ft20', 'ft24','ft48']
row = ['Never Takers', "Always Takers", 'Compliers']
summary_of_NAC = SimpleTable(summary_fractions, col, row, title =\
                            'The Fractions of Compliers, Always Takers and Never Takers In Months: 15,20,24,48')
print(summary_of_NAC)

The Fractions of Compliers, Always Takers and Never Takers In Months: 15,20,24,48
               ft15   ft20   ft24   ft48 
-----------------------------------------
Never Takers  0.7154 0.7245 0.7324 0.7158
Always Takers 0.1472 0.1609 0.1602 0.2337
Compliers     0.1374 0.1146 0.1074 0.0505
-----------------------------------------


2)

In [285]:

ft15d = df.dropna(subset = ['ft15'])
ft24d =df.dropna(subset = ['ft24'])
ft20d =df.dropna(subset = ['ft20'])
ft48d =df.dropna(subset = ['ft48'])
data = [ft15d, ft20d, ft24d, ft48d]
z = ['ft15','ft20','ft24','ft48']
for i,j in zip(data,z):
    for s in X:
        i[s + j] = i[s]*i[j]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


{imm,hsgrad,agelt25,age35p,working_at_baseline,anykidsu6,nevermarried}

In [286]:
X = ['imm', 'hsgrad','agelt25','age35p','working_at_baseline','anykidsu6','nevermarried']

In [287]:
result1 = {}
result15 = {}
result20 = {}
result24 = {}
result48 = {}
result = [result15,result20,result24,result48]

for i,p,x in zip(z,data, result):
    for j in X:
        reg = g.IV2SLS(p[j+i],p[i],sm.add_constant(p['treatment'])).fit()
        x[j] = reg.params[0]
    result1[i] = x

In [315]:
summl = pd.DataFrame(result1)
print(SimpleTable(summl.values.round(4),list(summl.columns),list(summl.index), title = 'Mean for Compliers for each Covariate'))

     Mean for Compliers for each Covariate     
                     ft15   ft20   ft24   ft48 
-----------------------------------------------
age35p              0.2775 0.292  0.2842 0.2891
agelt25             0.179  0.183  0.1925 0.1943
anykidsu6           0.5257 0.5281 0.5336 0.5268
hsgrad              0.5844 0.589  0.5953 0.6037
imm                 0.1088 0.1223 0.1224 0.1224
nevermarried        0.4798 0.4817 0.4809 0.5007
working_at_baseline 0.3663 0.3573 0.3521 0.3155
-----------------------------------------------
